# The Case of the Missing Period

In [1]:
import pandas as pd
import numpy as np
import random

from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
from astral import LocationInfo
from astral.sun import SUN

In [2]:
swell = pd.read_csv('data/swell.csv')
tide = pd.read_csv('data/tide.csv')
wind = pd.read_csv('data/wind.csv')
period = pd.read_csv('data/period.csv')

In [3]:
def checkHour(df):
    df['UTC'] = pd.to_datetime(df['UTC'], utc=True)
    onehour = timedelta(days=0, hours=1)
    baddies = []
    index = df.index.to_list()
    index.remove(0)
    for i in index:
        if (df['UTC'][i]-df['UTC'][i-1] != onehour):
            baddies.append(i)
    return len(baddies), baddies

Would expect number of nan values in merged df to equal the number of missing hours in the original dataset, plus any nan values from og dataset <br/>
Test this:

In [4]:
#number of missing hours in swell
(24*365*5+24) - swell.shape[0]

55

In [5]:
#number of nan values in swell
swell.isna().sum()

Height    0
Deg       2
UTC       0
dtype: int64

In [6]:
ts = tide.merge(swell, how='left', on='UTC')

In [7]:
ts.isna().sum()

UTC        0
Tide       0
Height    55
Deg       57
dtype: int64

that's what's expected!

In [8]:
#number of missing hours in period
(24*365*5+24) - period.shape[0]

57

In [9]:
#number of nan values in period
period.isna().sum()

UTC       0
Period    0
dtype: int64

In [10]:
tsp = ts.merge(period, how='left', on='UTC')

In [11]:
tsp.isna().sum()

UTC          0
Tide         0
Height      55
Deg         57
Period    8035
dtype: int64

nooooo

In [12]:
checkHour(period)

(4, [15403, 21538, 27029, 35007])

In [13]:
badi = checkHour(period)[1]


In [17]:
i=3
period.iloc[badi[i]-2:badi[i]+3]

,UTC,Period
35005,2020-12-31 22:30:00+00:00,7
35006,2020-12-31 23:30:00+00:00,4
35007,2021-01-01 00:00:00+00:00,7
35008,2021-01-01 01:00:00+00:00,15
35009,2021-01-01 02:00:00+00:00,15


Missing 7pm 10/4/2018 <br/>
Missing all of 6/18/2019 and the first 18 hours of 6/19 and the last 12 hours of 6/17 <br/>
**On 2/3/2020 at 2:30pm measurements are taken at the half hour instead of the hour** <br/>
**In the first hour of 2021, measurements go back to top of the hour** <br/>

SOooooo all those missing values in the merge are becasue almost a year's worth of period data is at the half hour, not the hour, so gotta change those measurements....not gonna worry about how the accuracy of data changes because only a 30min difference, not a big deal 

In [18]:
period.iloc[27029]

UTC       2020-02-03 14:30:00+00:00
Period                            9
Name: 27029, dtype: object

In [21]:
period['UTC'].dtype

datetime64[ns, UTC]

In [28]:
period['UTC'].iloc[27029:35007] = period['UTC'].iloc[27029:35007]-timedelta(minutes=30)

C:\Users\audre\AppData\Local\Temp\ipykernel_7068\876847691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period['UTC'].iloc[27029:35007] = period['UTC'].iloc[27029:35007]-timedelta(minutes=30)


In [29]:
checkHour(period)

(2, [15403, 21538])

These are the normal missing hours we already looked at! 

In [34]:
tsp = ts.merge(p, how='left', on='UTC')
tsp.isna().sum()

UTC        0
Tide       0
Height    55
Deg       57
Period    57
dtype: int64

In [33]:
p = pd.read_csv('data/period.csv')